## 1. Desempenho por Categoria

Nesta seção, queremos responder:

1. Quais categorias de produto apresentam maior contribuição para o faturamento mensal da empresa?
2. Em quais períodos (meses/estações) cada categoria performa melhor ou pior?
3. Quais categorias possuem maior potencial de crescimento além das malas, que hoje representam a maior fatia da receita?


In [0]:
%sql


--1.1 

SELECT
    p.DESC_GRUPO CATEGORIA,
    SUM(f.VALOR_VENDIDO) AS FATURAMENTO_TOTAL,
    SUM(f.QTD_VENDIDA) AS QTD_TOTAL
FROM database_mvp.gold.fato_vendas_mensal f
JOIN database_mvp.gold.dim_produto p
    ON f.SK_PRODUTO = p.SK_PRODUTO
JOIN database_mvp.gold.dim_loja l
    ON l.SK_LOJA = f.SK_LOJA
WHERE  l.TIPO_DESC <>'Importadora'
GROUP BY p.DESC_GRUPO
ORDER BY FATURAMENTO_TOTAL DESC;

In [0]:
%sql
-- 1.2 Em quais períodos (meses/anos) cada categoria performa melhor?

SELECT
    f.DATA_MES,
    p.DESC_GRUPO AS CATEGORIA,
    SUM(f.VALOR_VENDIDO) AS FATURAMENTO_MENSAL
FROM database_mvp.gold.fato_vendas_mensal f
JOIN database_mvp.gold.dim_produto p
    ON f.SK_PRODUTO = p.SK_PRODUTO
WHERE DATA_MES <> '2025-12-01'
GROUP BY f.DATA_MES, p.DESC_GRUPO
ORDER BY f.DATA_MES, CATEGORIA;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- 1.3 Categorias com maior potencial de crescimento (variação ao longo do tempo)

WITH mensal AS (
    SELECT
        f.DATA_MES,
        p.DESC_GRUPO AS CATEGORIA,
        SUM(f.VALOR_VENDIDO) AS FATURAMENTO
    FROM database_mvp.gold.fato_vendas_mensal f
    JOIN database_mvp.gold.dim_produto p
        ON f.SK_PRODUTO = p.SK_PRODUTO
    WHERE DATA_MES <> '2025-12-01' -- pois o mes nao esta completo.
    GROUP BY f.DATA_MES, p.DESC_GRUPO
)
SELECT
    CATEGORIA,
    AVG(FATURAMENTO) AS FAT_MEDIA_MENSAL,
    (MAX(FATURAMENTO) - MIN(FATURAMENTO)) AS VARIACAO_ABS,
    (MAX(FATURAMENTO) / NULLIF(MIN(FATURAMENTO),0)) AS TAXA_CRESCIMENTO_REL
FROM mensal
GROUP BY CATEGORIA
ORDER BY TAXA_CRESCIMENTO_REL DESC NULLS LAST;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql

-- 1.3 Categorias com maior potencial de crescimento (variação ao longo do tempo) - aprimorada; 
WITH mensal AS (
    SELECT
        f.DATA_MES AS DATA_MES,
        p.DESC_GRUPO AS CATEGORIA,
        SUM(f.VALOR_VENDIDO) AS FATURAMENTO
    FROM database_mvp.gold.fato_vendas_mensal f
    JOIN database_mvp.gold.dim_produto p
        ON f.SK_PRODUTO = p.SK_PRODUTO
    WHERE DATA_MES <> '2025-12-01' AND DESC_GRUPO NOT IN ('INSUMO', 'OUTROS', 'ALMOXARIFADO')
    GROUP BY f.DATA_MES, p.DESC_GRUPO
),
com_variacao AS (
    SELECT
        DATA_MES,
        CATEGORIA,
        FATURAMENTO,
        LAG(FATURAMENTO) OVER (
            PARTITION BY CATEGORIA
            ORDER BY DATA_MES
        ) AS FAT_ANTERIOR
    FROM mensal
)
SELECT
    DATA_MES,
    CATEGORIA,
    FATURAMENTO,
    FAT_ANTERIOR,
    (FATURAMENTO - FAT_ANTERIOR) AS DELTA_MENSAL,
    CASE 
        WHEN FAT_ANTERIOR IS NULL OR FAT_ANTERIOR = 0 THEN NULL
        ELSE round((FATURAMENTO - FAT_ANTERIOR) / FAT_ANTERIOR * 100,2)
    END AS CRESC_MENSAL_PCT
FROM com_variacao
ORDER BY CATEGORIA, DATA_MES;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- 1.3 Categorias com maior potencial de crescimento (variação ao longo do tempo) - aprofundando em cotidiano 
WITH mensal AS (
    SELECT
        f.DATA_MES AS DATA_MES,
        concat(p.DESC_GRUPO,'-', p.DESC_LINHA) AS CATEGORIA,
        SUM(f.VALOR_VENDIDO) AS FATURAMENTO
    FROM database_mvp.gold.fato_vendas_mensal f
    JOIN database_mvp.gold.dim_produto p
        ON f.SK_PRODUTO = p.SK_PRODUTO
    WHERE DATA_MES <> '2025-12-01' AND DESC_GRUPO IN ('COTIDIANO')
    GROUP BY f.DATA_MES, concat(p.DESC_GRUPO,'-', p.DESC_LINHA)
),
com_variacao AS (
    SELECT
        DATA_MES,
        CATEGORIA,
        FATURAMENTO,
        LAG(FATURAMENTO) OVER (
            PARTITION BY CATEGORIA
            ORDER BY DATA_MES
        ) AS FAT_ANTERIOR
    FROM mensal
)
SELECT
    DATA_MES,
    CATEGORIA,
    FATURAMENTO,
    FAT_ANTERIOR,
    (FATURAMENTO - FAT_ANTERIOR) AS DELTA_MENSAL,
    CASE 
        WHEN FAT_ANTERIOR IS NULL OR FAT_ANTERIOR = 0 THEN NULL
        ELSE round((FATURAMENTO - FAT_ANTERIOR) / FAT_ANTERIOR * 100,2)
    END AS CRESC_MENSAL_PCT
FROM com_variacao
ORDER BY CATEGORIA, DATA_MES;


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

## 2. Desempenho por Região

Objetivos desta seção:

1. Identificar quais estados demonstram maior abertura para categorias que não sejam apenas malas.
2. Avaliar em quais regiões a marca é percebida de forma mais ampla, consumindo outras categorias.


In [0]:
%sql
-- 2.1 Faturamento por categoria e estado

SELECT
    l.ESTADO,
    p.DESC_GRUPO AS CATEGORIA,
    SUM(f.VALOR_VENDIDO) AS FATURAMENTO
FROM database_mvp.gold.fato_vendas_mensal f
JOIN database_mvp.gold.dim_loja l
    ON f.SK_LOJA = l.SK_LOJA
JOIN database_mvp.gold.dim_produto p
    ON f.SK_PRODUTO = p.SK_PRODUTO
WHERE l.LOJA_ATIVA = true AND DESC_GRUPO NOT IN ('INSUMO', 'OUTROS', 'ALMOXARIFADO', 'MALAS')
GROUP BY l.ESTADO, p.DESC_GRUPO
ORDER BY l.ESTADO, FATURAMENTO DESC;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- 2.2 Participação de cada categoria no mix de faturamento por estado

WITH fat_total AS (
    SELECT
        l.ESTADO,
        SUM(f.VALOR_VENDIDO) AS FAT_ESTADO
    FROM database_mvp.gold.fato_vendas_mensal f
    JOIN database_mvp.gold.dim_loja l
        ON f.SK_LOJA = l.SK_LOJA
    WHERE l.LOJA_ATIVA = true
    GROUP BY l.ESTADO
),
fat_cat AS (
    SELECT
        l.ESTADO,
        p.DESC_GRUPO AS CATEGORIA,
        SUM(f.VALOR_VENDIDO) AS FAT_CATEGORIA
    FROM database_mvp.gold.fato_vendas_mensal f
    JOIN database_mvp.gold.dim_loja l
        ON f.SK_LOJA = l.SK_LOJA
    JOIN database_mvp.gold.dim_produto p
        ON f.SK_PRODUTO = p.SK_PRODUTO
    WHERE l.LOJA_ATIVA = true
    GROUP BY l.ESTADO, p.DESC_GRUPO
)
SELECT
    c.ESTADO,
    c.CATEGORIA,
    c.FAT_CATEGORIA,
    t.FAT_ESTADO,
    (c.FAT_CATEGORIA / t.FAT_ESTADO)*100 AS PARTICIPACAO_PERC
FROM fat_cat c
JOIN fat_total t
    ON c.ESTADO = t.ESTADO
ORDER BY PARTICIPACAO_PERC DESC;


Databricks visualization. Run in Databricks to view.

## 3. Eficiência Comercial (Reavaliação de Escopo)

Inicialmente, esta seção tinha como objetivo analisar métricas clássicas de eficiência comercial no varejo, tais como:

1. PA (Peças por Atendimento) médio por estado;
2. Diferenças de performance entre tipos de loja (ex.: rua vs shopping);
3. Identificação de lojas com maior variação mensal de vendas.

Entretanto, ao longo do desenvolvimento do MVP e da análise da base de dados disponível, identificou-se a ausência de informações fundamentais para o cálculo de algumas dessas métricas, tais como:

- número de atendimentos ou tickets por loja;
- classificação do tipo físico da loja (rua ou shopping).

Dessa forma, métricas como PA e ticket médio por atendimento não puderam ser calculadas de maneira consistente neste MVP, uma vez que sua estimação sem dados de atendimento poderia levar a interpretações incorretas.

Diante dessas limitações, optou-se por **reformular o foco analítico desta seção**, direcionando a análise para métricas plenamente suportadas pelos dados disponíveis e que ainda permitem gerar insights relevantes para o negócio.




## 3 Escala de vendas por loja (volume e valor)
Nesta subseção, o objetivo é aprofundar a análise do comportamento de vendas por loja, observando os seguintes aspectos:

- Quais lojas apresentam maior volume de peças vendidas?
- As lojas líderes em faturamento também lideram em quantidade?
- Existem lojas com alto volume e baixo valor médio por peça?
- Qual a participação das categorias não-malas no faturamento de cada loja?







In [0]:
%sql
-- 3.1 Quais lojas apresentam maior volume de peças vendidas?

SELECT
    l.NOME_LIMPO,
    l.ESTADO,
    SUM(f.QTD_VENDIDA) AS QTD_TOTAL
FROM database_mvp.gold.fato_vendas_mensal f
JOIN database_mvp.gold.dim_loja l
    ON f.SK_LOJA = l.SK_LOJA
GROUP BY l.NOME_LIMPO, l.ESTADO
ORDER BY QTD_TOTAL DESC
LIMIT 10;



In [0]:
%sql

--3.2 As lojas líderes em faturamento também lideram em quantidade?
WITH top_fat AS (
    SELECT
        l.NOME_LIMPO AS LOJA,
        l.ESTADO,
        SUM(f.VALOR_VENDIDO) AS FAT_TOTAL,
        SUM(f.QTD_VENDIDA) AS QTD_TOTAL
    FROM database_mvp.gold.fato_vendas_mensal f
    JOIN database_mvp.gold.dim_loja l
        ON f.SK_LOJA = l.SK_LOJA
    WHERE l.NOME_LIMPO <>'BAG FRANQUIAS'
    GROUP BY l.NOME_LIMPO, l.ESTADO
    ORDER BY FAT_TOTAL DESC
    LIMIT 15
),
top_qtd AS (
    SELECT
        l.NOME_LIMPO AS LOJA,
        l.ESTADO,
        SUM(f.VALOR_VENDIDO) AS FAT_TOTAL,
        SUM(f.QTD_VENDIDA) AS QTD_TOTAL
    FROM database_mvp.gold.fato_vendas_mensal f
    JOIN database_mvp.gold.dim_loja l
        ON f.SK_LOJA = l.SK_LOJA
    GROUP BY l.NOME_LIMPO, l.ESTADO
    ORDER BY QTD_TOTAL DESC
    LIMIT 15
)

SELECT
    LOJA,
    ESTADO,
    FAT_TOTAL,
    QTD_TOTAL
FROM (
    SELECT * FROM top_fat
    UNION ALL
    SELECT * FROM top_qtd
) t
GROUP BY LOJA, ESTADO, FAT_TOTAL, QTD_TOTAL
ORDER BY FAT_TOTAL DESC, QTD_TOTAL DESC;


Databricks visualization. Run in Databricks to view.

O gráfico evidencia que as lojas com maior faturamento não são necessariamente aquelas com maior volume de peças vendidas, indicando diferenças relevantes no mix de produtos comercializados. Observa-se a coexistência de lojas orientadas a itens de maior valor unitário e lojas com perfil de alto volume, que podem atuar como importantes vetores de captação e recorrência de clientes.

In [0]:
%sql

-- 3.3 Existem lojas com alto volume e baixo valor médio por peça?
SELECT
    l.NOME_LIMPO AS LOJAS,
    l.ESTADO,
    SUM(f.QTD_VENDIDA) AS QTD_TOTAL,
    SUM(f.VALOR_VENDIDO) AS FAT_TOTAL,
    CASE 
        WHEN SUM(f.QTD_VENDIDA) = 0 THEN NULL
        ELSE SUM(f.VALOR_VENDIDO) / SUM(f.QTD_VENDIDA)
    END AS VALOR_MEDIO_POR_PECA
FROM database_mvp.gold.fato_vendas_mensal f
JOIN database_mvp.gold.dim_loja l
    ON f.SK_LOJA = l.SK_LOJA
GROUP BY l.NOME_LIMPO, l.ESTADO
ORDER BY  QTD_TOTAL DESC, VALOR_MEDIO_POR_PECA ASC; 


A análise indica a existência de lojas com alto volume de peças vendidas e menor valor médio por item, caracterizando um perfil de vendas mais orientado a produtos de entrada e maior giro. Esse comportamento sugere que tais lojas podem desempenhar um papel estratégico na captação de clientes e aumento de recorrência, ainda que não sejam as principais contribuintes em faturamento total.

In [0]:
%sql
--- 3.4 Qual a participação das categorias não-malas no faturamento de cada loja?
WITH vendas_loja_categoria AS (
    SELECT
        l.NOME_LIMPO as LOJA,
        p.DESC_GRUPO AS CATEGORIA,
        SUM(f.VALOR_VENDIDO) AS FAT_CATEGORIA
    FROM database_mvp.gold.fato_vendas_mensal f
    JOIN database_mvp.gold.dim_loja l
        ON f.SK_LOJA = l.SK_LOJA
    JOIN database_mvp.gold.dim_produto p
        ON f.SK_PRODUTO = p.SK_PRODUTO
    WHERE l.TIPO IN ('A','P')
    GROUP BY l.NOME_LIMPO, p.DESC_GRUPO
),
total_loja AS (
    SELECT
        LOJA,
        SUM(FAT_CATEGORIA) AS FAT_TOTAL
    FROM vendas_loja_categoria
    GROUP BY LOJA
),
nao_malas AS (
    SELECT
        LOJA,
        SUM(FAT_CATEGORIA) AS FAT_NAO_MALAS
    FROM vendas_loja_categoria
    WHERE CATEGORIA <> 'MALAS'
    GROUP BY LOJA
)
SELECT
    t.LOJA,
    t.FAT_TOTAL,
    COALESCE(n.FAT_NAO_MALAS, 0) AS FAT_NAO_MALAS,
    COALESCE(n.FAT_NAO_MALAS, 0) / t.FAT_TOTAL AS PARTICIPACAO_NAO_MALAS
FROM total_loja t
LEFT JOIN nao_malas n
    ON t.LOJA = n.LOJA
ORDER BY PARTICIPACAO_NAO_MALAS DESC;
